<a href="https://colab.research.google.com/github/oz-e/applied-ml/blob/main/CuPL_ucf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Change the path if necessary
datasets_path = 'datasets'

import sys
import os

# Deploy AML code in colab
if 'google.colab' in sys.modules:
  if not os.path.exists('/content/applied-ml/'):
    !wget -q https://raw.githubusercontent.com/tsunrise/colab-github/main/colab_github.py
    import colab_github
    colab_github.github_auth(persistent_key=False)

    !git clone git@github.com:oz-e/applied-ml.git
    if not os.path.exists('/content/applied-ml/'):
      raise Exception('Please follow the instructions to add the SSH key to your account in order to clone private repo')
  %cd /content/applied-ml/

  # Install any other requirements (to be converted to requirements.txt)
  !pip install openai-clip

Looks that a private key is already created. If you have already push it to github, no action required.
 Otherwise, Please go to https://github.com/settings/ssh/new to upload the following key: 
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIFHmJoQTHvjDC5CFbH+0hyXoxIbBc64gmuePeSGWxhke root@c46c19f673a8

Please use SSH method to clone repo.
/content/applied-ml


In [14]:
import os

if not os.path.exists("CuPL"):
  !git clone https://github.com/sarahpratt/CuPL.git

In [15]:
import numpy as np
import torch
import clip
from pkg_resources import packaging
from CuPL.imagenet_prompts.standard_image_prompts import imagenet_templates
import pdb
from collections import defaultdict
from PIL import Image
import PIL
import json
from tqdm import tqdm

In [16]:
model, preprocess = clip.load("ViT-B/16")

In [17]:
import aml.datasets

all_images = aml.datasets.UCF101(datasets_path, split='test', transform=preprocess)
loader = torch.utils.data.DataLoader(all_images, batch_size=512, num_workers=8)

Downloading...
From (original): https://drive.google.com/uc?id=10Jqome3vtUA2keJkNanAiFpgbyC9Hc2O
From (redirected): https://drive.usercontent.google.com/download?id=10Jqome3vtUA2keJkNanAiFpgbyC9Hc2O&confirm=t&uuid=53a364d7-44ed-4431-b4bc-f8c090958e68
To: /content/applied-ml/datasets/ucf101/UCF-101-midframes.zip
100%|██████████| 140M/140M [00:02<00:00, 55.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1I0S0q91hJfsV9Gf4xDIjgDq4AqBNJb1y
To: /content/applied-ml/datasets/ucf101/split.json
100%|██████████| 1.56M/1.56M [00:00<00:00, 133MB/s]


In [18]:
PATH_TO_PROMPTS = './CuPL/all_prompts/full_prompts/ucf_prompts_full.json'

with open(PATH_TO_PROMPTS) as f:
  gpt3_prompts = json.load(f)

In [19]:
special_class_mapping = {
    'Hand Stand Pushups': 'Handstand_Pushups'
}

In [20]:
# For bridging classsname in dataset with name in CuLP prompts
new_gpt3_prompts = {}

for key, value in gpt3_prompts.items():
  new_key = special_class_mapping.get(key, key)
  new_gpt3_prompts[new_key.replace(' ', '_')] = value

In [21]:
def zeroshot_classifier(classnames, templates):
	with torch.no_grad():
		zeroshot_weights = []
		for classname in tqdm(classnames):
			texts = [template.format(classname) for template in templates] #format with class
			texts = clip.tokenize(texts).cuda() #tokenize
			class_embeddings = model.encode_text(texts) #embed with text encoder
			class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
			class_embedding = class_embeddings.mean(dim=0)
			class_embedding /= class_embedding.norm()
			zeroshot_weights.append(class_embedding)
		zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
	return zeroshot_weights

In [22]:
def zeroshot_classifier_gpt(classnames, templates, use_both):
	with torch.no_grad():
		zeroshot_weights = []
		for classname in tqdm(classnames):
			if use_both:
				texts = [template.format(classname) for template in templates]
			else:
				texts = []

			for t in new_gpt3_prompts[classname]:
				texts.append(t)
			texts = clip.tokenize(texts, truncate=True).cuda() #tokenize
			class_embeddings = model.encode_text(texts) #embed with text encoder
			class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
			class_embedding = class_embeddings.mean(dim=0)
			class_embedding /= class_embedding.norm()
			zeroshot_weights.append(class_embedding)

		zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
	return zeroshot_weights

In [23]:
print("\nCreating standard text embeddings...")
zeroshot_weights_base = zeroshot_classifier(all_images.classnames, imagenet_templates)
print("Done.\n")

print("Creating CuPL text embeddings...")
zeroshot_weights_cupl = zeroshot_classifier_gpt(all_images.classnames, imagenet_templates, False)
print("Done.\n")

print("Creating combined text embeddings...")
zeroshot_weights_gpt_both = zeroshot_classifier_gpt(all_images.classnames, imagenet_templates, True)
print("Done.\n")


Creating standard text embeddings...


100%|██████████| 101/101 [00:05<00:00, 17.93it/s]


Done.

Creating CuPL text embeddings...


100%|██████████| 101/101 [00:04<00:00, 25.15it/s]


Done.

Creating combined text embeddings...


100%|██████████| 101/101 [00:09<00:00, 11.02it/s]

Done.



In [24]:
total = 0.
correct_base = 0.
correct_cupl = 0.
correct_both = 0.

print("Classifying ImageNet...")

with torch.no_grad():

	for i, (images, target) in enumerate(tqdm(loader)):
		images = images.cuda()
		target = target.cuda()

		# predict
		image_features = model.encode_image(images)
		image_features /= image_features.norm(dim=-1, keepdim=True)

		logits_base = image_features @ zeroshot_weights_base
		logits_cupl = image_features @ zeroshot_weights_cupl
		logits_both = image_features @ zeroshot_weights_gpt_both

		pred_base = torch.argmax(logits_base, dim =1)
		pred_cupl = torch.argmax(logits_cupl, dim =1)
		pred_both = torch.argmax(logits_both, dim =1)

		for j in range(len(target)):
			total += 1.
			if pred_base[j] == target[j]:
				correct_base += 1.
			if pred_cupl[j] == target[j]:
				correct_cupl += 1.
			if pred_both[j] == target[j]:
				correct_both += 1.

Classifying ImageNet...


100%|██████████| 8/8 [00:29<00:00,  3.64s/it]


In [25]:
print()
top1 = (correct_base / total) * 100
print(f"Top-1 accuracy standard: {top1:.2f}")

top1 = (correct_cupl / total) * 100
print(f"Top-1 accuracy CuPL: {top1:.2f}")

top1 = (correct_both / total) * 100
print(f"Top-1 accuracy both: {top1:.2f}")


Top-1 accuracy standard: 65.08
Top-1 accuracy CuPL: 70.34
Top-1 accuracy both: 68.09
